In [3]:
from IPython.display import display
from app_widgets import AppWidgets

In [ ]:
# Widgets
app_widgets = AppWidgets("Measurement", "config.json")

# Populate dropdown lists
app_widgets.load_dropdown_lists()

# Measurement Uploader

## Select sample

In [ ]:
display(app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown_boxes)
app_widgets.create_button.on_click(app_widgets.upload_measurements_to_openbis)
app_widgets.quit_button.on_click(app_widgets.close_notebook)
app_widgets.samples_dropdown_boxes.children[0].observe(app_widgets.load_sample_metadata, names = 'value')

## Select measurements folders

In [ ]:
display(app_widgets.folder_selector)
display(app_widgets.bottom_buttons_hbox)
display(app_widgets.increase_buttons_size)